Implementation of the YOLOv8n model

In [ ]:
import os
import random
import shutil
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import supervision as sv
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

MAIN_PROJECT_DIR = "."  
DATA_DIR = os.path.join(MAIN_PROJECT_DIR, "data")
IMAGES_DIR = os.path.join(DATA_DIR, "images")
LABELS_DIR = os.path.join(DATA_DIR, "labels")

NATIVE_IMAGES_DIR = os.path.join(IMAGES_DIR, "native")
NATIVE_LABELS_DIR = os.path.join(LABELS_DIR, "native")

print(f"Main project directory: {MAIN_PROJECT_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Images directory: {IMAGES_DIR}")
print(f"Labels directory: {LABELS_DIR}")
print(f"Native Images directory: {NATIVE_IMAGES_DIR}")
print(f"Native Labels directory: {NATIVE_LABELS_DIR}")

assert os.path.exists(NATIVE_IMAGES_DIR), f"Directory not found: {NATIVE_IMAGES_DIR}"
assert os.path.exists(NATIVE_LABELS_DIR), f"Directory not found: {NATIVE_LABELS_DIR}"


Main project directory: .
Data directory: .\data
Images directory: .\data\images
Labels directory: .\data\labels
Native Images directory: .\data\images\native
Native Labels directory: .\data\labels\native


In [ ]:
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR = os.path.join(DATA_DIR, "val")
TEST_DIR = os.path.join(DATA_DIR, "test")

TRAIN_IMAGES_DIR = os.path.join(TRAIN_DIR, "images")
TRAIN_LABELS_DIR = os.path.join(TRAIN_DIR, "labels")
VAL_IMAGES_DIR = os.path.join(VAL_DIR, "images")
VAL_LABELS_DIR = os.path.join(VAL_DIR, "labels")
TEST_IMAGES_DIR = os.path.join(TEST_DIR, "images")
TEST_LABELS_DIR = os.path.join(TEST_DIR, "labels")

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_directory(TRAIN_DIR)
create_directory(VAL_DIR)
create_directory(TEST_DIR)
create_directory(TRAIN_IMAGES_DIR)
create_directory(TRAIN_LABELS_DIR)
create_directory(VAL_IMAGES_DIR)
create_directory(VAL_LABELS_DIR)
create_directory(TEST_IMAGES_DIR)
create_directory(TEST_LABELS_DIR)

image_files = sorted(
    [f for f in os.listdir(NATIVE_IMAGES_DIR) if f.endswith(".tif")]
)  
label_files = sorted([f for f in os.listdir(NATIVE_LABELS_DIR) if f.endswith(".txt")])

assert len(image_files) == len(
    label_files
), "The number of images and labels do not match."

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_index = int(len(image_files) * train_ratio)
val_index = int(len(image_files) * (train_ratio + val_ratio))

train_images = image_files[:train_index]
val_images = image_files[train_index:val_index]
test_images = image_files[val_index:]

train_labels = label_files[:train_index]
val_labels = label_files[train_index:val_index]
test_labels = label_files[val_index:]

def move_files(images, labels, dest_images_dir, dest_labels_dir):
    for image, label in zip(images, labels):
        shutil.copy(os.path.join(NATIVE_IMAGES_DIR, image), dest_images_dir)
        shutil.copy(os.path.join(NATIVE_LABELS_DIR, label), dest_labels_dir)

move_files(train_images, train_labels, TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR)
move_files(val_images, val_labels, VAL_IMAGES_DIR, VAL_LABELS_DIR)
move_files(test_images, test_labels, TEST_IMAGES_DIR, TEST_LABELS_DIR)

print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")
print(f"Number of test images: {len(test_images)}")


Number of training images: 2033
Number of validation images: 254
Number of test images: 255


In [ ]:
data_yaml_path = os.path.join(DATA_DIR, "data.yaml")
num_classes = 1 

with open(data_yaml_path, "w") as f:
    f.write(f"train: {TRAIN_IMAGES_DIR}\n")
    f.write(f"val: {VAL_IMAGES_DIR}\n")
    f.write(f"test: {TEST_IMAGES_DIR}\n")  
    f.write(f"nc: {num_classes}\n")
    f.write("names: ['solar_panel']\n")

print(f"data.yaml file created at: {data_yaml_path}")


data.yaml file created at: .\data\data.yaml


In [ ]:
model = YOLO("yolov8n.pt") 

results = model.train(
    data=data_yaml_path,
    epochs=100,  
    imgsz=416,
    val=True,
)  


Ultralytics 8.3.80  Python-3.11.5 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=.\data\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

100%|██████████| 755k/755k [00:00<00:00, 1.30MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning C:\Users\tarun\OneDrive\Desktop\Solar-Panel-Detection\data\train\labels... 2033 images, 0 backgrounds, 206 corrupt: 100%|██████████| 2033/2033 [00:08<00:00, 236.87it/s]

train: WARNING  C:\Users\tarun\OneDrive\Desktop\Solar-Panel-Detection\data\train\images\solarpanels_native_1__x0_0_y0_6963_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\tarun\OneDrive\Desktop\Solar-Panel-Detection\data\train\images\solarpanels_native_1__x0_0_y0_7119_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\tarun\OneDrive\Desktop\Solar-Panel-Detection\data\train\images\solarpanels_native_1__x0_0_y0_7229_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\tarun\OneDrive\Desktop\Solar-Panel-Detection\data\train\images\solarpanels_native_1__x0_0_y0_7230_dxdy_416.tif: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\Users\tarun\O

...

    100/100         0G     0.6494     0.4027     0.8524         62        416: 100%|██████████| 115/115 [04:22<00:00,  2.28s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:14<00:00,  1.86s/it]

                   all        242       1683      0.816       0.63      0.727      0.488



100 epochs completed in 8.220 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 6.2MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics 8.3.80  Python-3.11.5 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:15<00:00,  1.92s/it]


                   all        242       1683       0.82      0.629      0.726      0.489
Speed: 0.7ms preprocess, 50.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train3


In [ ]:
best_model = YOLO("runs/detect/train3/weights/best.pt")  

def visualize_predictions(model, image_path, label_path):
    
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape

    gt_boxes = []
    with open(label_path, "r") as f:
        
        for line in f.readlines():
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x1 = int((x_center - width / 2) * w)
            y1 = int((y_center - height / 2) * h)
            x2 = int((x_center + width / 2) * w)
            y2 = int((y_center + height / 2) * h)
            gt_boxes.append([x1, y1, x2, y2])

    gt_boxes = np.array(gt_boxes)
    results = model.predict(image)

    if len(results) > 0:
        result = results[0] 
        if result.boxes:  
            boxes = result.boxes.xyxy.cpu().numpy()  
            confidence = result.boxes.conf.cpu().numpy()  
            class_id = result.boxes.cls.cpu().numpy().astype(int)  

            detections = sv.Detections(
                xyxy=boxes,
                confidence=confidence,
                class_id=class_id,
            )

            gt_annotator = sv.BoundingBoxAnnotator(
                color=sv.Color.GREEN, thickness=2
            )
            pred_annotator = sv.BoundingBoxAnnotator(
                color=sv.Color.RED, thickness=2
            )

            annotated_image = image.copy()
            if len(gt_boxes) > 0:
                gt_detections = sv.Detections(
                    xyxy=gt_boxes, confidence=np.ones(len(gt_boxes)), class_id=np.zeros(len(gt_boxes))
                )  
                annotated_image = gt_annotator.annotate(scene=annotated_image, detections=gt_detections)

            if detections is not None:
                annotated_image = pred_annotator.annotate(scene=annotated_image, detections=detections)

            plt.figure(figsize=(10, 10))
            plt.imshow(annotated_image)
            plt.axis("off")
            plt.show()
        else:
            print("No detections found.")
    else:
        print("No prediction results.")

num_samples = 3
random_indices = random.sample(range(len(test_images)), num_samples)

for i in random_indices:
    image_file = test_images[i]
    label_file = test_labels[i]
    image_path = os.path.join(TEST_IMAGES_DIR, image_file)
    label_path = os.path.join(TEST_LABELS_DIR, label_file)
    visualize_predictions(best_model, image_path, label_path)



0: 416x416 1 solar_panel, 111.3ms
Speed: 5.1ms preprocess, 111.3ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 416)


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


<Figure size 1000x1000 with 1 Axes>


0: 416x416 8 solar_panels, 113.0ms
Speed: 2.1ms preprocess, 113.0ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 416)


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


<Figure size 1000x1000 with 1 Axes>


0: 416x416 6 solar_panels, 157.3ms
Speed: 1.6ms preprocess, 157.3ms inference, 1.7ms postprocess per image at shape (1, 3, 416, 416)


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


<Figure size 1000x1000 with 1 Axes>

In [ ]:
test_image_paths = [
    os.path.join(TEST_IMAGES_DIR, filename) for filename in test_images
]
test_label_paths = [
    os.path.join(TEST_LABELS_DIR, filename) for filename in test_labels
]


def load_ground_truth_boxes(label_path, image_width, image_height):
    gt_boxes = []
    with open(label_path, "r") as f:
        for line in f.readlines():
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x1 = (x_center - width / 2) * image_width
            y1 = (y_center - height / 2) * image_height
            x2 = (x_center + width / 2) * image_width
            y2 = (y_center + height / 2) * image_height
            gt_boxes.append([x1, y1, x2, y2])
    return np.array(gt_boxes)

all_ground_truth_boxes = []
all_predicted_boxes = []
all_confidence_scores = []

for image_path, label_path in zip(test_image_paths, test_label_paths):
    image = cv2.imread(image_path)
    h, w, _ = image.shape

    gt_boxes = load_ground_truth_boxes(label_path, w, h)
    all_ground_truth_boxes.append(gt_boxes)

    results = best_model.predict(image)

    if len(results) > 0:
        result = results[0]  
        if result.boxes:  
            boxes = result.boxes.xyxy.cpu().numpy()  
            confidence = result.boxes.conf.cpu().numpy()  
            class_id = result.boxes.cls.cpu().numpy().astype(int)  
            all_predicted_boxes.append(boxes)
            all_confidence_scores.append(confidence)
        else:
            all_predicted_boxes.append(np.empty((0, 4)))  
            all_confidence_scores.append(np.empty(0))  
    else:
        all_predicted_boxes.append(np.empty((0, 4)))  
        all_confidence_scores.append(np.empty(0))  

from sklearn.metrics import average_precision_score

import numpy as np

def compute_iou(box1, box2):
    """Compute IoU between two boxes."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0


def compute_map50(gt_boxes_list, pred_boxes_list, confidence_scores_list, iou_threshold=0.5):

    all_precisions = []

    for gt_boxes, pred_boxes, confidences in zip(gt_boxes_list, pred_boxes_list, confidence_scores_list):
        if len(pred_boxes) == 0 or len(gt_boxes) == 0:
            all_precisions.append(0)
            continue
        sorted_indices = np.argsort(-confidences)
        pred_boxes = pred_boxes[sorted_indices]
        confidences = confidences[sorted_indices]

        tp = np.zeros(len(pred_boxes))
        fp = np.zeros(len(pred_boxes))
        matched_gt = set()

        for i, pred_box in enumerate(pred_boxes):
            best_iou = 0
            best_gt_idx = -1

            for j, gt_box in enumerate(gt_boxes):
                if j in matched_gt:
                    continue
                iou = compute_iou(pred_box, gt_box)
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = j

            if best_iou >= iou_threshold:
                tp[i] = 1
                matched_gt.add(best_gt_idx)
            else:
                fp[i] = 1

        tp_cumsum = np.cumsum(tp)
        fp_cumsum = np.cumsum(fp)

        precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)
        recalls = tp_cumsum / (len(gt_boxes) + 1e-6)

        precisions = np.concatenate(([0], precisions, [0]))
        recalls = np.concatenate(([0], recalls, [1]))

        precisions = np.maximum.accumulate(precisions[::-1])[::-1]

        ap = np.sum((recalls[1:] - recalls[:-1]) * precisions[1:])
        all_precisions.append(ap)

    return np.mean(all_precisions)

mAP50 = compute_map50(
    all_ground_truth_boxes,
    all_predicted_boxes,
    all_confidence_scores,
    iou_threshold=0.5,
)

print(f"mAP50: {mAP50}")

import supervision as sv

ground_truth_detections = [
    sv.Detections(
        xyxy=gt_boxes,
        class_id=np.zeros(len(gt_boxes), dtype=int)  
    )
    for gt_boxes in all_ground_truth_boxes
]

predicted_detections = [
    sv.Detections(
        xyxy=pred_boxes,
        confidence=confidences,
        class_id=np.zeros(len(pred_boxes), dtype=int)  
    )
    for pred_boxes, confidences in zip(all_predicted_boxes, all_confidence_scores)
]

class_names = ["solar_panel"] 

confusion_matrix = sv.ConfusionMatrix.from_detections(
    predictions=predicted_detections,
    targets=ground_truth_detections,
    classes=class_names,
)

def compute_prf(conf_matrix):
    tp = conf_matrix.matrix[0, 0]  
    fp = conf_matrix.matrix[0, 1:].sum()  
    fn = conf_matrix.matrix[1:, 0].sum()  
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = (
        2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    )

    return precision, recall, f1_score

iou_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]
for iou_threshold in iou_thresholds:
    confusion_matrix.iou_threshold = iou_threshold
    precision, recall, f1 = compute_prf(confusion_matrix)
    print(
        f"IoU={iou_threshold}: Precision={precision:.2f}, Recall={recall:.2f}, F1-Score={f1:.2f}"
    )


0: 416x416 3 solar_panels, 81.1ms
Speed: 5.6ms preprocess, 81.1ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 5 solar_panels, 58.5ms
Speed: 1.2ms preprocess, 58.5ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 4 solar_panels, 55.9ms
Speed: 0.8ms preprocess, 55.9ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 5 solar_panels, 55.0ms
Speed: 1.4ms preprocess, 55.0ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 3 solar_panels, 58.6ms
Speed: 1.2ms preprocess, 58.6ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 5 solar_panels, 56.3ms
Speed: 0.8ms preprocess, 56.3ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 3 solar_panels, 126.8ms
Speed: 1.3ms preprocess, 126.8ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 6 solar_panels, 69.6ms
Speed: 1.3ms preprocess, 69.6ms inferenc

In [ ]:
import supervision as sv

def create_prf_table(iou_thresholds, confidence_thresholds):
    table = []
    for iou_threshold in iou_thresholds:
        row = []
        for confidence_threshold in confidence_thresholds:
            
            ground_truth_detections = [
                sv.Detections(
                    xyxy=gt_boxes,
                    class_id=np.zeros(len(gt_boxes), dtype=int)  
                )
                for gt_boxes in all_ground_truth_boxes
            ]

            predicted_detections = [
                sv.Detections(
                    xyxy=pred_boxes,
                    confidence=confidences,
                    class_id=np.zeros(len(pred_boxes), dtype=int) 
                )
                for pred_boxes, confidences in zip(all_predicted_boxes, all_confidence_scores)
            ]


            filtered_predicted_detections = []
            for pred_detection in predicted_detections:
                filtered_boxes = pred_detection.xyxy[pred_detection.confidence >= confidence_threshold]
                filtered_confidences = pred_detection.confidence[pred_detection.confidence >= confidence_threshold]
                filtered_class_id = np.zeros(len(filtered_boxes), dtype=int)
                filtered_predicted_detections.append(
                    sv.Detections(xyxy=filtered_boxes, confidence=filtered_confidences, class_id=filtered_class_id)
                )

            
            class_names = ["solar_panel"]  
            confusion_matrix = sv.ConfusionMatrix.from_detections(
                predictions=filtered_predicted_detections,
                targets=ground_truth_detections,
                classes=class_names,
                iou_threshold=iou_threshold,
            )

            precision, recall, f1 = compute_prf(confusion_matrix)
            row.append((precision, recall, f1))
        table.append(row)

    return table

iou_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]
confidence_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]

prf_table = create_prf_table(iou_thresholds, confidence_thresholds)

print("Precision, Recall, F1-score Table:")
for i, row in enumerate(prf_table):
    print(f"IoU={iou_thresholds[i]}")
    for j, (precision, recall, f1) in enumerate(row):
        print(
            f"  Confidence={confidence_thresholds[j]}: Precision={precision}, Recall={recall}, F1={f1}"
        )


Precision, Recall, F1-score Table:
IoU=0.1
  Confidence=0.1: Precision=0.7801992528019925, Recall=0.7905362776025237, F1=0.7853337511751801
  Confidence=0.3: Precision=0.7801992528019925, Recall=0.7905362776025237, F1=0.7853337511751801
  Confidence=0.5: Precision=0.701120797011208, Recall=0.8790007806401249, F1=0.7800484932455836
  Confidence=0.7: Precision=0.5610211706102117, Recall=0.95042194092827, F1=0.7055599060297572
  Confidence=0.9: Precision=0.09464508094645081, Recall=1.0, F1=0.17292377701934017
IoU=0.3
  Confidence=0.1: Precision=0.7752179327521793, Recall=0.7854889589905363, F1=0.7803196490128487
  Confidence=0.3: Precision=0.7752179327521793, Recall=0.7854889589905363, F1=0.7803196490128487
  Confidence=0.5: Precision=0.699252801992528, Recall=0.8766588602654176, F1=0.7779702112919986
  Confidence=0.7: Precision=0.5603985056039851, Recall=0.9493670886075949, F1=0.7047768206734534
  Confidence=0.9: Precision=0.09464508094645081, Recall=1.0, F1=0.17292377701934017
IoU=0.5
 